# Задание 1

(**NB.** для запуска примеров кода нужен Python версии не ниже **3.10**, допускается использование других версий, в этом случае нужно самостоятельно избавиться от конструкции `match`).

Есть следующий код для [автоматического дифференцирования](https://en.wikipedia.org/wiki/Automatic_differentiation), в котором используются особенности системы типов языка `Python`: 

In [1]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)    

    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__
 

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d 

Поддерживаются две операции - сложение и умножение. Применить можно так:

In [2]:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

f_diff = diff(f)

# значение производной в точке x = 2
f_diff(2)

22.0

## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__) 
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

In [3]:
import math
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number

@dataclass
class Dual:
    value: float
    d: float
        
    def __add__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)    

    __rmul__ = __mul__  
    __radd__ = __add__  
       
    def __neg__(self) -> "Dual":
        return Dual(-1 * self.value, -1 * self.d)

    def __sub__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value - o_value, self.d - o_d)
            case Number():
                return Dual(self.value - float(other), self.d)

    def __rsub__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(o_value - self.value, o_d - self.d)
            case Number():
                return Dual(float(other) - self.value, -self.d)
        
    def __truediv__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(value=self.value/o_value, d=(self.d*o_value - o_d*self.value)/(o_value**2))
            case Number():
                return Dual(self.value / float(other), self.d / float(other))
        
    def __pow__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(value=self.value ** o_value,d=(self.value ** o_value)*(o_d * math.log(self.value) + self.d * o_value / self.value))
            case Number():
                return Dual(self.value ** float(other), float(other) * (self.value ** (other - 1.)) * self.d)
     
    def __rpow__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(value=other.value ** self.value,d=(o_value ** self.value)*(self.d * math.log(o_value) + o_d * self.value / o_alue))
            case Number():
                return Dual(float(other) ** self.value, math.log(float(other)) * (float(other) ** self.value) * self.d)
    
def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d


In [4]:
# test
funcs1 = [
    lambda x: 2 * x + 1,
    lambda x: x ** 5 + 3 * x + 1,
    lambda x: (5 * x * x + x ** 10 + 10 ** (x)) / (2 * x - 50 * x * x)
]
for func in funcs1:
    print(diff(func)(2))

2.0
83.0
-21.50298489750842


## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

In [5]:
def exp(x: Union["Dual", Number]) -> "Dual":
    match x:
        case Dual(x.value, x.d):
            return Dual(math.exp(x.value), math.exp(x.value) * x.d)
        case Number():
            return Dual(math.exp(x))    

def cos(x: Union["Dual", Number]) -> "Dual":
    match x:
        case Dual(x.value, x.d):
            return Dual(math.cos(x.value), -math.sin(x.value) * x.d)        
        case Number():
            return Dual(math.cos(x))

def sin(x: Union["Dual", Number]) -> "Dual":
    match x:
        case Dual(x.value, x.d):
            return Dual(math.sin(x.value), math.cos(x.value) * x.d)
        case Number():
            return Dual(math.sin(x))
    

def log(x: Union["Dual", Number], base: float = math.e) -> "Dual":
    match x:
        case Dual(x.value, x.d):
            return Dual(math.log(x.value, base), x.d / (x.value * math.log(base)))
        case Number():
            return Dual(math.log(x, base), 0)

In [6]:
# test
funcs2 = [
    lambda x: exp(2 * x),
    lambda x: log(5 * (2 * x ** x + 3)),
    lambda x: (sin(x) + 2 * cos(x) ** 2) / (exp(x) + 3 * x)
]
for func in funcs2:
    print(diff(func)(2))

109.19630006628847
1.2313797676799603
0.009197858981707774


## Задание 1.3 (3 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+5 баллов**)

In [7]:
from scipy.misc import derivative

def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

derivative(f, 2.)

22.0

In [8]:
def derivative_1(func: Callable[[Dual], Dual], x: float, eps: float = 1e-4) -> float:
    return (func(Dual(x + eps,1.)).value - func(Dual(x - eps,1.)).value) / (2 * eps)

In [9]:
# test
funcs3 = [
    lambda x: exp(2 * x),
    lambda x: log(5 * (2 * x ** x + 3)),
    lambda x: (sin(x) + 2 * cos(x) ** 2) / (exp(x) + 3 * x)
]
for func in funcs3:
    print(abs(derivative_1(func, 2) - diff(func)(2)))

7.280989962055173e-07
1.9429724495978462e-10
1.0514090709179413e-09


## Задание 1.4 (10 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках 

Генерацию случайных функций можно осуществить, например, двумя путями. 
1. Генерировать функцию в текстовом виде, зачем использовать встроенную функцию [eval](https://docs.python.org/3/library/functions.html#eval)

```python
func = eval("lambda x: 2 * x + 5")
assert func(42) == 89 
```

2. Использовать стандартный модуль [ast](https://docs.python.org/3/library/ast.html), который позволяет во время выполнения программы манипулировать [Абстрактным Синтаксическим Деревом](https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%81%D1%82%D1%80%D0%B0%D0%BA%D1%82%D0%BD%D0%BE%D0%B5_%D1%81%D0%B8%D0%BD%D1%82%D0%B0%D0%BA%D1%81%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE).
Например, выражение 

```python
func = lambda x: 2 * x + 5
```

Можно запрограммировать с помощью кода:

```python

expr = ast.Expression(
    body=ast.Lambda(
        args=ast.arguments(
            args=[
                ast.arg(arg='x')
            ],
            posonlyargs=[],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]
        ),
        body=ast.BinOp(
            left=ast.BinOp(
                left=ast.Constant(value=2),
                op=ast.Mult(),
                right=ast.Name(id='x', ctx=ast.Load())
            ),
            op=ast.Add(),
            right=ast.Constant(value=5)
        )
    )
)

ast.fix_missing_locations(expr)

func = eval(compile(expr, filename="", mode="eval"))

assert func(42) == 89
```

При реализации нужно учитывать области допустимых значений функций.

In [10]:
import math
import random
from itertools import repeat
from typing import Generic, List, Dict, Tuple, TypeVar, Union, Callable, Protocol

class RandomFunc:
    Functions: Dict[Callable[..., str], int] = {
        lambda x: f'cos({x})': 1,
        lambda x: f'sin({x})': 1,
        lambda x: f'exp({x})': 1,
        lambda x, y: f'({x} + {y})': 2,
        lambda x, y: f'({x} - {y})': 2,
        lambda x, y: f'({x} * {y})': 2
    }


    def __init__(self, var: str, cmin: int = 1, cmax: int = 10) -> None:
        self.constant = lambda: str(random.randint(cmin, cmax))
        self.variable = lambda: var

    def randleaf(self) -> str:
        if random.random() <= 0.5:
            return self.variable()
        return self.constant()

    
    def randfun(self) -> Tuple[Callable[..., str], int]:
        return random.choice(list(self.Functions.items()))

    def generate(self, maxlevel: int) -> str:
        if maxlevel == 0:
            return self.randleaf()
        func, nparam = self.randfun()
        params = [self.generate(maxlevel - 1) for _ in range(nparam)]
        return func(*params)

    def __call__(self, maxlevel: int) -> str:
        return self.generate(maxlevel)


In [11]:
def test(n: int, level: int = 3):
    generator = RandomFunc('x')
    for _ in range(n):
        expr = generator(maxlevel=level)
        func = eval(f'lambda x: {expr}')
        x = 2
        try:
            ratio = abs(derivative_1(func, x) - func(Dual(x, 1)).d)
        except:
            continue
        print(f'{expr}, [{x}], {ratio}')

test(n=10)

(cos((x * 3)) * exp((x * x))), [2], 1.6853811644068628e-06
(cos((2 + x)) + sin((x * x))), [2], 9.872791717491225e-08
exp(((2 - x) - exp(x))), [2], 4.241613015681933e-10
cos((sin(x) + (x + 5))), [2], 5.06092612262421e-10
cos(sin((x + x))), [2], 2.292541056991837e-08


## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

In [12]:
T = TypeVar('T')

class NCallable(Protocol, Generic[T]):
    def __call__(self, *args: T) -> T:
        pass

def ndiff(func: NCallable[Dual]) -> Callable[..., List[float]]:
    def identity(n: int) -> List[List[int]]:
        return [[(0, 1)[i == j] for i in range(n)] for j in range(n)]

    def wrap(*vars_: float) -> List[float]:
        nvalue, nd = repeat(vars_, len(vars_)), identity(len(vars_))
        result: List[float] = []
        for values, ds in zip(nvalue, nd):
            params: List[Dual] = []
            for value, d in zip(values, ds):
                params.append(Dual(value, d))
            result.append(func(*params).d)
        return result

    return wrap

In [15]:
def nfunc(x, y, z):
    return x * y + z - 5 * y
assert ndiff(nfunc)(10, 10, 10) == [10., 5., 1.]